<a href="https://colab.research.google.com/github/custom-hyper/Analytics/blob/main/Journaling_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Setup Environment

In [ ]:
#@title Connect to FNS { display-mode: "code" }
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ccxt --quiet
!pip install scikit-optimize --quiet
!pip install pytrends --quiet
!pip install pycoingecko --quiet
!pip install pandas --quiet
!pip install requests --quiet
!pip install pipreqs --quiet
!pip install configparser --quiet
!pip install psycopg2-binary==2.8.6 --quiet
!pip install pandas_ta --quiet
!pip install backtesting --quiet
!pip install bokeh --quiet
!pip install matplotlib --quiet
!pip install mplfinance --quiet
!pip install dydx-python --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.12.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.7/384.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... d

In [ ]:
#@title Ta-lib – Extra steps { display-mode: "code" }
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

     ━━━━━━━━

In [ ]:
#@title Import Project  Dependencies { display-mode: "code" }

import talib as ta
import pandas_ta as ta
from google.colab import data_table



#data science
import pandas as pd
import matplotlib.pyplot as plt
#from pandas.io.json import json_normalize
import time
import json

#Visualization
import mplfinance as fplt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
# datasets
from pycoingecko import CoinGeckoAPI
#OS
import requests
from datetime import datetime as dt, timedelta, date
import logging
import sys
from tqdm.notebook import tqdm
import os

#e/database
import sqlite3 as sq
import psycopg2 as pg
import configparser
from sqlalchemy import create_engine
import hashlib

#Backtesting
from backtesting import Strategy
from backtesting import Backtest
import ccxt



/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
#@title  Connect SQLite { display-mode: "code" }
path = '/content/drive/MyDrive/workspace/pipelines/db/' #@param {type: 'string'
database = "db.sqlite" #@param {type: 'string'
conn_SQLite = sq.connect(path+database)

# Data Extraction –> load into DB

## Interactive Brokers

In [ ]:
path = '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/IB/'
csv = 'Trader_Synch.csv'

data = pd.read_csv(path + csv)
df_stocks = pd.DataFrame(data)

df_stocks

,ClientAccountID,AccountAlias,Model,CurrencyPrimary,FXRateToBase,AssetClass,SubCategory,Symbol,Description,Conid,...,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83
0,U10465221,TFSA,NaN,CAD,0.72730,STK,COMMON,COIN,TOKENS.COM CORP,484596250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,U10465221,TFSA,NaN,CAD,0.72730,STK,COMMON,TRBE,TRIBE PROPERTY TECHNOLOGIES,478647500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U10465221,TFSA,NaN,CAD,0.73588,STK,COMMON,COIN,TOKENS.COM CORP,484596250,...,LMT,NaN,N,0.0,NaN,NaN,NaN,NaN,0.0,0.0
3,U10465221,TFSA,NaN,CAD,0.73588,STK,COMMON,COIN,TOKENS.COM CORP,484596250,...,LMT,NaN,N,0.0,NaN,NaN,NaN,NaN,0.0,0.0
4,U10465221,TFSA,NaN,CAD,0.73588,STK,COMMON,COIN,TOKENS.COM CORP,484596250,...,LMT,NaN,N,0.0,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,U10465221,TFSA,NaN,CAD,0.74658,CASH,NaN,USD.CAD,USD.CAD,15016062,...,MKT,NaN,N,0.0,NaN,NaN,NaN,NaN,0.0,0.0
754,U10465221,TFSA,NaN,CAD,0.74044,CASH,NaN,USD.CAD,USD.CAD,15016062,...,MKT,NaN,N,0.0,NaN,NaN,NaN,NaN,0.0,0.0
755,U10465221,TFSA,NaN,CAD,0.74171,CASH,NaN,USD.CAD,USD.CAD,15016062,...,MKT,NaN,N,0.0,NaN,NaN,NaN,NaN,0.0,0.0
756,U10465221,TFSA,NaN,CAD,0.74171,CASH,NaN,USD.CAD,USD.CAD,15016062,...,MKT,NaN,N,0.0,NaN,NaN,NaN,NaN,0.0,0.0


## Blockpit

In [ ]:
import os
import pandas as pd

# Set the directory path
directory_path = '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/Blockpit/raw/'

# List all CSV files in the directory
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

# Initialize an empty list to store DataFrames
dfs = []

# Read each CSV file and append its DataFrame to the list
for csv_file in csv_files:
    file_path = os.path.join(directory_path, csv_file)
    try:
        # Specify delimiter as semicolon
        data = pd.read_csv(file_path, delimiter=';')
        dfs.append(data)
    except Exception as e:
        print(f"Error reading {file_path}: {str(e)}")

# Concatenate all DataFrames into a single DataFrame
df_merged = pd.concat(dfs, ignore_index=True)
# Convert Timestamp to datetime object
df_merged['Timestamp'] = pd.to_datetime(df_merged['Timestamp'], format='%d.%m.%Y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')



# Conditionally create the currency_pair column only for 'Trade' transactions
df_merged['currency_pair'] = ''
df_merged.loc[df_merged['Transaction Type'] == 'Trade', 'currency_pair'] = df_merged['Outgoing Asset'] + '/' + df_merged['Incoming Asset']

# Calculate currency_price only for 'Trade' transactions
df_merged['currency_price'] = ''
df_merged.loc[df_merged['Transaction Type'] == 'Trade', 'currency_price'] = df_merged['Incoming Amount'] / df_merged['Outgoing Amount']

# Replace 'USDC' with 'USDT' and append 'USDT' if missing
df_merged['currency_pair'] = df_merged['currency_pair'].str.replace('/USDC', '/USDT')
df_merged['currency_pair'] = df_merged['currency_pair'].str.replace('USDC/', '')
df_merged['currency_pair'] = df_merged['currency_pair'].str.replace('USDT/', '')

# Handling missing or empty strings and appending 'USDT'
def append_usdt(x):
    if not x or pd.isnull(x):
        return 'USDT'
    elif 'USDT' not in x:
        return x + '/USDT'
    else:
        return x

df_merged['currency_pair'] = df_merged['currency_pair'].apply(append_usdt)
# Create the 'USD_outgoing' column and populate it with outgoing amount if outgoing asset contains 'USD'
df_merged['USD_outgoing'] = df_merged.apply(lambda row: row['Outgoing Amount']*-1 if isinstance(row['Outgoing Asset'], str) and 'USD' in row['Outgoing Asset'] else 0, axis=1)
# Create the 'USD_outgoing' column and populate it with outgoing amount if outgoing asset contains 'USD'
df_merged['USD_incoming'] = df_merged.apply(lambda row: row['Incoming Amount'] if isinstance(row['Incoming Asset'], str) and 'USD' in row['Incoming Asset'] else 0, axis=1)

#Calculate order P&L
df_merged['PnL'] = df_merged['USD_incoming'] + df_merged['USD_outgoing']

# Create the 'order' column and set it to 'buy' for 'USDT' or 'USDC' outgoing assets in 'Trade' transactions
df_merged['order'] = ''
df_merged.loc[(df_merged['Transaction Type'] == 'Trade') & (df_merged['Outgoing Asset'].isin(['USDT', 'USDC'])), 'order'] = 'buy'


df_merged.loc[(df_merged['Transaction Type'] == 'Trade') & (df_merged['Incoming Asset'].isin(['USDT', 'USDC'])), 'order'] = 'sell'
df_merged['proprietary_ID'] = ''



P&L calculation, creating a column that only looks at USD or USDT and if it's a by its and inflow it fits a sell it's an outflow


In [ ]:

# Create the 'USD_outgoing' column and populate it with outgoing amount if outgoing asset contains 'USD'
df_merged['USD_outgoing'] = df_merged.apply(lambda row: row['Outgoing Amount']*-1 if isinstance(row['Outgoing Asset'], str) and 'USD' in row['Outgoing Asset'] else 0, axis=1)
# Create the 'USD_outgoing' column and populate it with outgoing amount if outgoing asset contains 'USD'
df_merged['USD_incoming'] = df_merged.apply(lambda row: row['Incoming Amount'] if isinstance(row['Incoming Asset'], str) and 'USD' in row['Incoming Asset'] else 0, axis=1)

#Calculate order P&L
df_merged['PnL'] = df_merged['USD_incoming'] + df_merged['USD_outgoing']

df_merged
#USD_incoming

merged_csv_path = os.path.join( '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/Blockpit/merged_file.csv')
df_merged.to_csv(merged_csv_path, index=False)

# Now df_merged contains all the data from the CSV files in the specified directory

df_merged


,Blockpit ID,Timestamp,Source Type,Source Name,Integration,Transaction Type,Outgoing Asset,Outgoing Amount,Incoming Asset,Incoming Amount,...,Transaction ID,Note,Merge ID,currency_pair,currency_price,USD_outgoing,USD_incoming,PnL,order,proprietary_ID
0,239723866350421540,2023-01-01 03:19:17,Chain,Polygon,Ethereum (0x0...d60),NonTaxableIn,NaN,NaN,ETHStation 1672539544077,1.000000,...,0xfbf364b08a7232ce9b2bc1a00de7c8f7ba7cec7b00cc...,Fallback,NaN,USDT,,0.000000,0.000000,0.000000,,
1,239723900416558639,2023-01-17 10:56:18,Chain,Polygon,Ethereum (0x0...d60),NonTaxableIn,NaN,NaN,Reward Token. Claim reward at https://musd.tech,518.000000,...,0x9413125b3f92cc31401c0b041702f6ac9df39758af46...,Fallback,NaN,USDT,,0.000000,0.000000,0.000000,,
2,239723836570863122,2023-01-23 03:45:59,Chain,Ethereum,Ethereum (0x0...d60),NonTaxableIn,NaN,NaN,ETH,1.167051,...,0x9a9537b1c0e902e0b3dbf051b728e550d5b7b1e29a4a...,Native_Transfer,NaN,USDT,,0.000000,0.000000,0.000000,,
3,239723836570863121,2023-01-23 03:55:35,Chain,Ethereum,Ethereum (0x0...d60),Withdrawal,USDC,42278.373999,NaN,NaN,...,0x799b2c8498cbd162813b6cafd22f6083d392c273d02f...,ERC20_Transfer,2.397229e+17,USDT,,-42278.373999,0.000000,-42278.373999,,
4,239722923043062974,2023-01-23 03:55:35,Chain,Ethereum,Hot (0xD...E13),Deposit,NaN,NaN,USDC,42278.373999,...,0x799b2c8498cbd162813b6cafd22f6083d392c273d02f...,ERC20_Transfer,2.397238e+17,USDT,,0.000000,42278.373999,42278.373999,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,285407645236464622,2024-03-16 03:30:45,API,Coinbase,Coinbase,Trade,USDC,21843.667840,ORCA,4759.300000,...,NaN,Limit Order,NaN,ORCA/USDT,0.21788,-21843.667840,0.000000,-21843.667840,buy,
1588,285381827743450494,2024-03-16 09:13:37,Chain,Arbitrum,Hot (0xD...E13),NonTaxableIn,NaN,NaN,&Earn esgmx rewards at https://esgmx.io,425.000000,...,0xa59990c18d5498c89fcd40d68682527475e9713334b9...,Fallback,NaN,USDT,,0.000000,0.000000,0.000000,,
1589,285389845851802452,2024-03-16 13:37:59,API,MEXC Global,MEXC Global,Trade,USDT,43.977600,INJ,0.960000,...,NaN,NaN,NaN,INJ/USDT,0.021829,-43.977600,0.000000,-43.977600,buy,
1590,285389845868579669,2024-03-16 13:38:15,API,MEXC Global,MEXC Global,Trade,USDT,3945.157200,INJ,86.120000,...,NaN,NaN,NaN,INJ/USDT,0.021829,-3945.157200,0.000000,-3945.157200,buy,


## Bitrue

## Crypto.com

In [ ]:

path = '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/crypto.com/'
csv = 'transactions.csv'

data = pd.read_csv(path + csv)
df_crypto = pd.DataFrame(data)
# Convert Timestamp to datetime object and reformat it to SQL formatting (Eastern Standard Time)
df_crypto['Timestamp'] = pd.to_datetime(df_crypto['Timestamp (UTC)'], format='%m/%d/%Y %H:%M').dt.tz_localize('UTC').dt.tz_convert('EST').dt.strftime('%Y-%m-%d %H:%M:%S')
#Create currency_pair using USDT
# Create currency_pair column based on Transaction Kind
# Create currency_pair column based on Transaction Kind
df_crypto['currency_pair'] = df_crypto.apply(lambda row: row['To Currency'] + "/USDT" if row['Transaction Kind'] == 'viban_purchase' else (row['Currency'] + "/USDT" if row['Transaction Kind'] == 'crypto_exchange' else row['Currency'] + "/USDT"), axis=1)
df_crypto['order'] = df_crypto.apply(lambda row: "buy" if row['Transaction Kind'] == 'viban_purchase' else ("sell" if row['Transaction Kind'] == 'crypto_exchange' else "sell"), axis=1)
#create P and L and put a positive sign went crypto exchange and negative when viban_purchase
# Create P&L column

# Create P&L column
df_crypto['PnL'] = df_crypto.apply(lambda row: -row['Native Amount (in USD)'] if isinstance(row['To Currency'], str) and 'USD' not in row['To Currency'] else row['Native Amount (in USD)'], axis=1)



# Create currency_pair column based on Transaction Kind

#Reformat timestamp using EST time zone
df_crypto['Source Name'] = 'crypto.com'
df_crypto['proprietary_ID'] = ''
df_crypto


,Timestamp (UTC),Transaction Description,Currency,Amount,To Currency,To Amount,Native Currency,Native Amount,Native Amount (in USD),Transaction Kind,Transaction Hash,Timestamp,currency_pair,order,PnL,Source Name,proprietary_ID
0,2/14/2024 18:15,CAD -> BONK,CAD,-1500.00,BONK,7.744830e+07,USD,1091.188832,1091.188832,viban_purchase,NaN,2024-02-14 13:15:00,BONK/USDT,buy,-1091.188832,crypto.com,
1,2/11/2024 17:57,CAD -> BONK,CAD,-3000.01,BONK,1.631618e+08,USD,2209.801908,2209.801908,viban_purchase,NaN,2024-02-11 12:57:00,BONK/USDT,buy,-2209.801908,crypto.com,
2,2/10/2024 0:22,Withdraw USDC (Polygon (native USDC only)),USDC,-2176.04,NaN,NaN,USD,2162.778341,2162.778341,crypto_withdrawal,0x98ac0a189eba3de30cb079d96e2043904cc525adb974...,2024-02-09 19:22:00,USDC/USDT,sell,2162.778341,crypto.com,
3,2/10/2024 0:15,CAD -> USDC,CAD,-3000.00,USDC,2.170560e+03,USD,2196.725279,2196.725279,viban_purchase,NaN,2024-02-09 19:15:00,USDC/USDT,buy,2196.725279,crypto.com,
4,1/13/2024 18:42,Withdraw 1INCH (ERC20),1INCH,-7410.10,NaN,NaN,USD,3487.674225,3487.674225,crypto_withdrawal,0x7b184586d1362940fdc3bfba211014e8e36cb6d5d57c...,2024-01-13 13:42:00,1INCH/USDT,sell,3487.674225,crypto.com,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,10/22/2023 0:33,CAD -> FIL,CAD,-4200.00,FIL,8.787250e+02,CAD,4200.000000,3140.179176,viban_purchase,NaN,2023-10-21 19:33:00,FIL/USDT,buy,-3140.179176,crypto.com,
57,10/6/2023 23:27,CAD -> SNX,CAD,-3000.00,SNX,1.045189e+03,CAD,3000.000000,2242.985126,viban_purchase,NaN,2023-10-06 18:27:00,SNX/USDT,buy,-2242.985126,crypto.com,
58,10/1/2023 22:47,Withdraw USDC (ERC20),USDC,-10613.07,NaN,NaN,CAD,14211.695430,10625.540490,crypto_withdrawal,0x63fc5975b58e6a89c4233c33197fc6643cc6d91b6786...,2023-10-01 17:47:00,USDC/USDT,sell,10625.540490,crypto.com,
59,10/1/2023 22:44,CAD -> USDC,CAD,-3000.01,USDC,2.162300e+03,CAD,3000.010000,2242.992602,viban_purchase,NaN,2023-10-01 17:44:00,USDC/USDT,buy,2242.992602,crypto.com,


In [ ]:
# Merge the DataFrames based on a common column (e.g., index) while keeping all rows from both DataFrames
orders_power_bi = pd.merge(df_merged, df_crypto, how='outer')
orders_power_bi.head()


,Blockpit ID,Timestamp,Source Type,Source Name,Integration,Transaction Type,Outgoing Asset,Outgoing Amount,Incoming Asset,Incoming Amount,...,Transaction Description,Currency,Amount,To Currency,To Amount,Native Currency,Native Amount,Native Amount (in USD),Transaction Kind,Transaction Hash
0,2.397239e+17,2023-01-01 03:19:17,Chain,Polygon,Ethereum (0x0...d60),NonTaxableIn,NaN,NaN,ETHStation 1672539544077,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.397239e+17,2023-01-17 10:56:18,Chain,Polygon,Ethereum (0x0...d60),NonTaxableIn,NaN,NaN,Reward Token. Claim reward at https://musd.tech,518.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.397238e+17,2023-01-23 03:45:59,Chain,Ethereum,Ethereum (0x0...d60),NonTaxableIn,NaN,NaN,ETH,1.167051,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.397238e+17,2023-01-23 03:55:35,Chain,Ethereum,Ethereum (0x0...d60),Withdrawal,USDC,42278.373999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.397229e+17,2023-01-23 03:55:35,Chain,Ethereum,Hot (0xD...E13),Deposit,NaN,NaN,USDC,42278.373999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
orders = "orders_power_bi"

orders_power_bi.to_sql(orders, conn_SQLite, if_exists='replace', index=False) # writes to file
conn_SQLite.commit()


In [ ]:
orders_power_bi_path = os.path.join( '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/')
orders_power_bi.to_csv(orders_power_bi_path, index=False)

IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/'

## DYDX

In [ ]:

import os
import glob
import pandas as pd
os.chdir('/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/DYDX/')

path = '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/DYDX/'
csv = 'raw_file.csv'

data = pd.read_csv(path + csv)
df_new = pd.DataFrame(data)

df_new['UNIX'] = [int(time.mktime(time.strptime(date.rstrip('Z'), '%Y-%m-%dT%H:%M:%S.%f'))) for date in df_new['createdAt'].tolist()]
df_new['UNIX'] = df_new['UNIX']*1000
df_new['createdAt'] = pd.to_datetime(df_new['createdAt'])
df_new['createdAt'] = df_new['createdAt'].dt.strftime('%m/%d/%Y, %H:%M:%S')

df_new['size'] = df_new.apply(lambda row: row['size'] * (-1 if row['side'] == 'BUY'else 1), axis=1)
df_new['Total'] = df_new.apply(lambda row: row['price'] * row['size'] , axis=1)
df_new['Total_net_fees'] = df_new.apply(lambda row: row['Total']- row['fee'] , axis=1)
df_new['Fee Currency'] = 'USD'
df_new['market'] = df_new['market'].str.replace("USD", "USDT")
df_new['market'] = df_new['market'].str.replace("-", "/")
df_new['Exchange'] = 'dydx'
df_new['Proprietary_ID'] = ''

df_new['Unique_ID'] = df_new.apply(lambda row: ''.join(str(row[col]) for col in df_new.columns if col != 'Proprietary_ID'), axis=1)
df_new['Unique_ID'] = df_new['Unique_ID'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())

df_new.rename(columns={'createdAt': 'Time', 'side': 'Side', 'price': 'Price','type': 'Type', 'size': 'Size', 'fee': 'Fee', 'market': 'Market'}, inplace=True)


df_new.head()

## FTX

In [ ]:

import os
import glob
import pandas as pd
os.chdir("/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/FTX/")

path = '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/FTX/'
path_sav = '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/FTX/merged/'
csv = 'merged_trades.csv'
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( path_sav+csv, index=False, encoding='utf-8-sig')


## Questrade

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/Questrade/Activities_for_01Jun2020_to_09Jul2023.xlsx'
df = pd.read_excel(file_path)
df.tail()


In [ ]:
import pandas as pd
import datetime
import hashlib

file_path = '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/Questrade/Activities_for_01Jun2020_to_09Jul2023.xlsx'
df = pd.read_excel(file_path)

# Convert "Double-click" and "Settlement Date" columns to datetime objects
df['Double-click'] = pd.to_datetime(df['Double-click'])
df['Settlement Date'] = pd.to_datetime(df['Settlement Date'])

# Calculate holding duration and create a unique ID for each trade
holding_duration = df['Settlement Date'] - df['Double-click']
df['Holding Duration'] = holding_duration

def generate_unique_id(row):
    unique_string = str(row.values)
    return hashlib.sha1(unique_string.encode()).hexdigest()

df['Unique ID'] = df.apply(generate_unique_id, axis=1)

# Display the updated DataFrame
df.tail()


In [ ]:
Table = "Questrade_proprietary_transaction_history"
df.to_sql(Table, conn_SQLite, if_exists='replace', index=False) # writes to file results gives

In [ ]:

path = '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/FTX/merged/'
csv = 'merged_trades.csv'

data = pd.read_csv(path + csv)
df = pd.DataFrame(data)

df['Market'] = df['Market'].str.replace("USDT", "USD")
df['Market'] = df['Market'].str.replace("USD", "USDT")
df['Market'] = df['Market'].str.replace("-PERP", "/USDT")

df['Exchange'] = 'ftx'
df['UNIX'] = [int(time.mktime(time.strptime(date, '%d/%m/%Y, %H:%M:%S'))) for date in df['Time'].tolist()]
df['UNIX'] = df['UNIX']*1000
df['Sie'] = df.apply(lambda row: row['Size'] * (-1 if row['Side'] == 'buy'else 1), axis=1)
df['Side'] = df['Side'].str.upper()
df['Total'] = df.apply(lambda row: row['Price'] * row['Size'] , axis=1)
df.rename(columns={'Order Type': 'Type'}, inplace=True)

df['buy_signal'] = df[df['Side'] == 'BUY']['Price']
df['sell_signal'] = df[df['Side'] == 'SELL']['Price']

df.tail()


## load to DB

In [ ]:
path_sav = "/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/DYDX/"
csv = "preprocessed_pre-ID.csv"
df_new.to_csv( path_sav+csv, index=False, encoding='utf-8-sig')

NameError: name 'df_new' is not defined

# Create Trade ID

## Load new files

In [ ]:
df_new.tail()

## Load old file  with IDs

In [ ]:
#@title
##@title Load Trade ID Journal { display-mode: "code" }

path = "/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/"
csv = "trade_history_proprietary_ID.csv"


data = pd.read_csv(path + csv)
df_Proprietary_ID = pd.DataFrame(data)
#df_Proprietary_ID["Market"] = [word + "/USDT" for word in df_Proprietary_ID["Market"]]
df_Proprietary_ID[df_Proprietary_ID['Proprietary_ID'] != '']
df_Proprietary_ID['Market'] = df_Proprietary_ID['Market'].str.replace("/USDT", "")
df_Proprietary_ID['Market'] = df_Proprietary_ID['Market']+"/USDT"
df_Proprietary_ID['Unique_ID'] = df_Proprietary_ID.apply(lambda row: ''.join(str(row[col]) for col in df_Proprietary_ID.columns if col != 'Proprietary_ID'), axis=1)
df_Proprietary_ID['Unique_ID'] = df_Proprietary_ID['Unique_ID'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
df_Proprietary_ID.tail()


In [ ]:
# Step 1: Find the row with the highest Unix timestamp
#latest_row = df_Proprietary_ID.loc[df_Proprietary_ID['UNIX'].idxmax()]

# Find the latest date in the list
latest_date = max(df_Proprietary_ID['Time'])

# Format the latest date as a string
#latest_date_str = latest_date.strftime("%m/%d/%Y, %H:%M:%S")

print("Latest date:", latest_date)

## Append new fille to old

## Load History

In [ ]:
# Filter rows from the df_new
filtered_rows = df_new[df_new['Time'] > latest_date]

# Concatenate df_Proprietary_ID and filtered_rows to form the target table
target_table = pd.concat([df_Proprietary_ID, filtered_rows], ignore_index=True)
# Alternatively, if you want to modify the 'target_table' in place, you can use the 'inplace' parameter:
# target_table.append(filtered_rows, ignore_index=True, inplace=True)

# Display the updated target_table

path_sav = "/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/UPDATED_trade_Journal/"
current_date = pd.to_datetime('today').strftime('%Y-%m-%d_%H-%M-%S')
# Generate the filename with the current date
output_filename = f'trade_histtory_proprietary_ID_{current_date}.csv'

target_table.to_csv( path_sav+ output_filename, index=False, encoding='utf-8-sig')

In [ ]:
path = '/content/drive/MyDrive/workspace/pipelines/datasets/trading_journal/LOAD – report/trade_histtory_proprietary_ID_2023-07-20.csv'

df = pd.read_csv(path)
df.tail(1)

In [ ]:
#@title
import pandas as pd

# Example DataFrame


# Initialize the trade ID
trade_id = 1

# Initialize the current sign (1 for positive, -1 for negative, 0 for neutral)
current_sign = 0

# Initialize the list to store the trade IDs
df['Trade ID'] = 0


for Exchange in df['Exchange'].unique():

    Filtered = df[(df['Exchange'] == Exchange)].sort_values(by='Time')
    #trade = trade.reset_i_ndex(drop=True)
    for Market in Filtered['Market'].unique():


        #df['Cumulative Size'] = df['Size'].cumsum()
        df['cumulative_total_net_of_fees'] = df['Total_net_fees'].cumsum()



In [ ]:
#@title
import pandas as pd
from datetime import datetime

# Read the dfset into a pandas dfFrame
df = df

# Convert the 'Time' column to datetime format
df['Time'] = pd.to_datetime(df['Time'])

# Sort the dfFrame by 'Proprietary_ID' and 'Time' in ascending order
df = df.sort_values(['Proprietary_ID', 'Time'], ascending=[True, True])

# Initialize new columns for Entry Date, Close Date, and Holding Duration
df['Entry Date'] = pd.NaT
df['Close Date'] = pd.NaT
df['Holding Duration'] = pd.NaT

# Iterate over each unique Proprietary_ID
for proprietary_id in df['Proprietary_ID'].unique():
    # Filter the df for the current Proprietary_ID
    trade_df = df[df['Proprietary_ID'] == proprietary_id]

    # Extract the earliest and latest timestamps for the trade
    entry_date = trade_df['Time'].min()
    close_date = trade_df['Time'].max()

    # Calculate the holding duration
    holding_duration = close_date - entry_date
    holding_duration_days = holding_duration.total_seconds() / (60 * 60 * 24)
    # Update the corresponding columns for the trade
    # Update the corresponding columns for the trade
    df.loc[df['Proprietary_ID'] == proprietary_id, 'Entry Date'] = entry_date
    df.loc[df['Proprietary_ID'] == proprietary_id, 'Close Date'] = close_date
    df.loc[df['Proprietary_ID'] == proprietary_id, 'Holding Duration'] = holding_duration
    df.loc[df['Proprietary_ID'] == proprietary_id, 'Holding Duration (Days)'] = holding_duration_days


In [ ]:
df = df.sort_values(by='Time')
df['cumulative_total_net_of_fees'] = df['Total_net_fees'].cumsum()
df['Total_fees'] = df['Fee'].cumsum()
df_s = df.reset_index(drop=True)
df.tail(1)

## Create long/short row

In [ ]:


# Assuming your DataFrame is named 'df'
# Sort the DataFrame by Proprietary_ID and Time in ascending order
df.sort_values(by=['Proprietary_ID', 'Time'], ascending=True, inplace=True)

# Create a function to categorize the trade type
def categorize_trade(row):
    if row['Side'] == 'BUY':
        return 'long'
    elif row['Side'] == 'SELL':
        return 'short'
    else:
        return 'Unknown'

# Group the DataFrame by Proprietary_ID and get the earliest timestamp for each group
earliest_timestamp = df.groupby('Proprietary_ID')['Time'].first()

# Create a dictionary to map Proprietary_ID to the trade type (long or short)
proprietary_id_trade_type = {}
for proprietary_id, time in earliest_timestamp.items():
    trade_type = df[(df['Proprietary_ID'] == proprietary_id) & (df['Time'] == time)].apply(categorize_trade, axis=1).values[0]
    proprietary_id_trade_type[proprietary_id] = trade_type

# Use the map function to create the 'Trade Type' column based on the Proprietary_ID trade type mapping
df['Trade Type'] = df['Proprietary_ID'].map(proprietary_id_trade_type)

# Display the updated DataFrame with the 'Trade Type' column
df


# Historical Prices

In [ ]:
#can create list of takers with order entry

#extract entire historical price using hourly candlestick and volume

#Extract entire timeframe on CoinGecko

#Extract entire trade report data

#Performance Report

## Select Historical Trades

## Main Statistics

In [ ]:

total_before_fees = df['Total'].sum()
print( "total before fees: ", total_before_fees)
fees =df['Fee'].sum()
print("total fees: ", fees)
PL = df['Total_net_fees'].cumsum()
print("P&L: ", PL)





In [ ]:
# Group the DataFrame by 'Proprietary_ID' and calculate the sum of 'Total Net of Fees'
#trade_report = df.groupby('Proprietary_ID')['Total_net_fees'].sum()

trade_report = df.groupby('Proprietary_ID').agg({ 'Market': 'first', 'Trade Type':'first','Total_net_fees': 'sum','Holding Duration (Days)':'last','Entry Date':'last','Close Date':'last','Size': 'sum'})


In [ ]:
trade_report = trade_report.sort_values('Total_net_fees', ascending=False)
trade_report

## Generating Positions Error Report

In [ ]:
error_report = trade_report[(trade_report['Size'] > 0.000001) | (trade_report['Size'] < -0.000001)]
error_report.to_csv('error_report.csv')
error_report.sort_values('Market', ascending=True)

In [ ]:
import matplotlib.pyplot as plt

# Set the dark theme for the plots
plt.style.use('dark_background')



# Separate long and short positions data along with market names
long_data = [(pnl, market) for pnl, market, trade_type in zip(trade_report["Total_net_fees"], trade_report["Market"], trade_report["Trade Type"]) if trade_type == 'long']
short_data = [(pnl, market) for pnl, market, trade_type in zip(trade_report["Total_net_fees"], trade_report["Market"], trade_report["Trade Type"]) if trade_type == 'short']

# Create a new figure with the desired size for long positions chart
plt.figure(figsize=(20, 8))

# Create a bar chart for long positions
plt.bar(range(len(long_data)), [pnl for pnl, _ in long_data], color=['green' if pnl >= 0 else 'red' for pnl, _ in long_data])

# Add market names as tick labels
plt.xticks(range(len(long_data)), [market for _, market in long_data], rotation=45, ha='right')

# Add labels and title for long positions chart
plt.xlabel('Market')
plt.ylabel('P&L')
plt.title('P&L per Long Trade')

# Display the plot for long positions chart
plt.show()

# Create a new figure with the desired size for short positions chart
plt.figure(figsize=(12, 8))

# Create a bar chart for short positions
plt.bar(range(len(short_data)), [pnl for pnl, _ in short_data], color=['green' if pnl >= 0 else 'red' for pnl, _ in short_data])

# Add market names as tick labels
plt.xticks(range(len(short_data)), [market for _, market in short_data], rotation=45, ha='right')

# Add labels and title for short positions chart
plt.xlabel('Market')
plt.ylabel('P&L')
plt.title('P&L per Short Trade')

# Display the plot for short positions chart
plt.show()


## P&L over time

In [ ]:
symbol = df['Market'].unique()
symbol

## P&L per trade

## P&L versus holding duration

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
# Set the dark theme for the plot
plt.style.use('dark_background')
plt.figure(figsize=(12, 8))

holding_period = trade_report["Holding Duration (Days)"]
pnl = trade_report["Total_net_fees"]  # y-axis data

# Define colors based on P&L values (green for positive and red for negative)
colors = ['green' if p >= 0 else 'red' for p in pnl]

# Create scatterplot with transparency and custom colors
plt.scatter(holding_period, pnl, c=colors, alpha=0.6)

# Add labels and title
plt.xlabel('Holding Period (Days)')
plt.ylabel('P&L')
plt.title('Scatterplot: Holding Period vs P&L')
# Set x-axis to log scale
# Set x-axis to log scale
plt.xscale('log')

# Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Set the dark theme for the plots
plt.style.use('dark_background')


# Separate long and short positions data along with their holding periods and P&L
long_data = [(holding_period, pnl) for holding_period, pnl, trade_type in zip(trade_report["Holding Duration (Days)"], trade_report["Total_net_fees"], trade_report["Trade Type"]) if trade_type == 'long']
short_data = [(holding_period, pnl) for holding_period, pnl, trade_type in zip(trade_report["Holding Duration (Days)"], trade_report["Total_net_fees"], trade_report["Trade Type"]) if trade_type == 'short']

# Create a new figure with the desired size for long positions chart
plt.figure(figsize=(12, 8))

# Define colors based on P&L values (green for positive and red for negative) for long positions chart
colors_long = ['green' if pnl >= 0 else 'red' for _, pnl in long_data]

# Create scatterplot with transparency and custom colors for long positions chart
plt.scatter([holding_period for holding_period, _ in long_data], [pnl for _, pnl in long_data], c=colors_long, alpha=0.6)

# Add labels and title for long positions chart
plt.xlabel('Holding Period (Days)')
plt.ylabel('P&L')
plt.title('Scatterplot: Holding Period vs P&L for Long Positions')
# Set x-axis to log scale
plt.xscale('log')

# Display the plot for long positions chart
plt.show()

# Create a new figure with the desired size for short positions chart
plt.figure(figsize=(12, 8))

# Define colors based on P&L values (green for positive and red for negative) for short positions chart
colors_short = ['green' if pnl >= 0 else 'red' for _, pnl in short_data]

# Create scatterplot with transparency and custom colors for short positions chart
plt.scatter([holding_period for holding_period, _ in short_data], [pnl for _, pnl in short_data], c=colors_short, alpha=0.6)

# Add labels and title for short positions chart
plt.xlabel('Holding Period (Days)')
plt.ylabel('P&L')
plt.title('Scatterplot: Holding Period vs P&L for Short Positions')
# Set x-axis to log scale
plt.xscale('log')

# Display the plot for short positions chart
plt.show()


## P&L per symbol

In [ ]:
trade_report_symbol = df.groupby('Market').agg({'Total_net_fees': 'sum','Entry Date':'last','Close Date':'last','Holding Duration (Days)':'last'})
trade_report_symbol

In [ ]:

# Sort the DataFrame by the 'Date' column (if it's not already sorted)
trade_report_symbol = trade_report_symbol.sort_values('Total_net_fees')
pnl_values = trade_report_symbol['Total_net_fees']
colors = ['red' if pnl < 0 else 'green' for pnl in pnl_values]
# Extract data for the line chart
dates = trade_report_symbol.index

# Sort the DataFrame by the 'P&L' column (smallest to largest)

# Create the line chart
plt.figure(figsize=(10, 6))
plt.bar(dates, pnl_values,color=colors)
plt.xlabel('Market')
plt.ylabel('P&L')
plt.title('Profit and Loss (P&L) Over Time')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Display the line chart
plt.show()

 Trade Review

## P&L per tag

In [ ]:
# @title Default title text
Event = 2 #@param {type:"integer"}
id =  17

exch = 'kucoin'
exchange = getattr(ccxt,exch)()
exchange.load_markets()
pair = df_catalyst['ccxt_pair'][id]
signal = df_catalyst['Catalyst Datetime'][id]
catalyst = df_catalyst['Specific Catalyst Category'][id]
CoinGecko_id = df_catalyst['CoinGecko ID'][id]

index = 'ETH/USDT'

In [ ]:
id=3
exchange = 'kucoin'
pair = 'UNI'

In [ ]:
sig = pd.Timestamp(signal).timestamp()*1000
sig = int(sig)

In [ ]:

# Assuming your dataset has a column named 'proprietary ID' (replace with the actual column name if different)
proprietary_id_to_find = 1
filtered_data = df[df['Proprietary_ID'] == proprietary_id_to_find]

# Now, filtered_data contains all the rows where 'proprietary ID' is equal to 1.
# You can access the 'data' column of these rows using:
resulting_data = filtered_data['data']

# If the 'data' column contains text data separated by spaces, you can split it into a list like this:
data_list = resulting_data.str.split().tolist()

# The data_list will contain the data associated with 'proprietary ID 1' as lists within the list.
# For example, if 'data' had values 'A B C' and 'X Y Z' for 'proprietary ID 1', the data_list would be [['A', 'B', 'C'], ['X', 'Y', 'Z']].


In [ ]:
data = exchange.fetchOHLCV(pair, '1m', since = (sig-9000))
df_1min = pd.DataFrame(data, columns = ['date','open','high','low','close','volume'])

df_1min['date'] = pd.to_datetime(df_1min['date'], unit='ms')

df_1min = df_1min.set_index('date')

# Resample the dataframe to create the desired interval
df_5min = df_1min.resample('5T').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})
df_1hour = df_1min.resample('1H').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})
df_1day = df_1min.resample('1D').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})

df_5min = df_5min.reset_index()
df_1hour = df_1hour.reset_index()
df_1day = df_1day.reset_index()
df_1min = df_1min.reset_index()

df_1min['returns'] = df_1min['close'].pct_change()
# Calculate the cumulative returns
df_1min['cumulative_returns'] = (1 + df_1min['returns']).cumprod()
df_1min.tail(2)

In [ ]:
# Calculate the cumulative returns
df_1min['cumulative_returns'] = (1 + df_1min['returns']).cumprod()

# Plot the equity curve
import matplotlib.pyplot as plt
plt.plot(df_1min['cumulative_returns'])
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.title('Equity Curve')
plt.show()

In [ ]:
data = exchange.fetchOHLCV(pair, '1d', since = (sig-9000))
df_1day= pd.DataFrame(data, columns = ['date','open','high','low','close','volume'])

df_1day['date'] = pd.to_datetime(df_1day['date'], unit='ms')
df_1day['returns'] = df_1day['close'].pct_change()
df_1day.tail(2)

In [ ]:
# Calculate the cumulative returns
df_1day['cumulative_returns'] = (1 + df_1day['returns']).cumprod()

# Plot the equity curve
import matplotlib.pyplot as plt
plt.plot(df_1day['cumulative_returns'])
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.title('Equity Curve')
plt.show()

In [ ]:
df_1day['sma10']=ta.sma(df_1day.close, length=10)
df_1day['sma20']=ta.sma(df_1day.close, length=20)
df_1day['sma50']=ta.sma(df_1day.close, length=50)
df_1day['sma100']=ta.sma(df_1day.close, length= 100)
df_1day['sma200']=ta.sma(df_1day.close, length=200)

df_1day = df_1day.reset_index()
df_1day.tail(2)

In [ ]:
my_bbands = ta.bbands(df_1hour.close, length=14, std=2.0)
my_bbands[0:20]
df_1hour=df_1hour.join(my_bbands)

df_1hour['sma10']=ta.sma(df_1hour.close, length=10)
df_1hour['sma20']=ta.sma(df_1hour.close, length=20)
df_1hour['sma50']=ta.sma(df_1hour.close, length=50)
df_1hour['sma100']=ta.sma(df_1hour.close, length= 100)
df_1hour['sma200']=ta.sma(df_1hour.close, length=200)

df_1hour.tail(2)

In [ ]:
import pandas_ta as ta

trace1 = {
'x': df_1day.date,
'open': df_1day.open,
'close': df_1day.close,
'high': df_1day.high,
'low': df_1day.low,
'type': 'candlestick',
'name': 'symbol' ,
'showlegend': True
}


trace2 = {
    'x': df_1day.date,
    'y': df_1day.sma10,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'yellow'
            },
    'name': 'SMA 10'
}

trace3 = {
    'x': df_1day.date,
    'y': df_1day.sma20,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'orange'
    },
    'name': 'SMA 20'
}
trace4 = {
    'x': df_1day.date,
    'y': df_1day.sma50,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'green'
            },
    'name': 'SMA 50'
}

trace5 = {
    'x': df_1day.date,
    'y': df_1day.sma100,
     'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'green'
            },
    'name': 'SMA 100'
}
trace6 = {
'x': df_1day.date,
    'y': df_1day.sma200,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'purple'
    },
    'name': 'SMA 200'
}

data = [trace1, trace2, trace3,trace4,trace5,trace6]

# Config graph layout



  #CHARTING
fig = go.Figure(data=data)

#fig.update_xaxes(rangeslider_visible=False)

fig.update_layout(
    title='test',
    autosize=False, width= 1000, height=1000,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="black", template ='plotly_dark',

    )



fig.add_scatter(x= Trade.Time, y= Trade['buy_signal'], mode="markers",
                marker=dict(size= 10, color="white"),
                name="Signal")

fig.add_scatter(x= Trade.Time, y= Trade['sell_signal'], mode="markers",
                marker=dict(size= 10, color="yellow"),
                name="Signal")
fig.show()






In [ ]:


trace1 = {
'x': df_1hour.date,
'open': df_1hour.open,
'close': df_1hour.close,
'high': df_1hour.high,
'low': df_1hour.low,
'type': 'candlestick',
'name': 'symbol' ,
'showlegend': True
}


trace2 = {
    'x': df_1hour.date,
    'y': df_1hour.sma10,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'yellow'
            },
    'name': 'SMA 10'
}

trace3 = {
    'x': df_1hour.date,
    'y': df_1hour.sma20,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'orange'
    },
    'name': 'SMA 20'
}
trace4 = {
    'x': df_1hour.date,
    'y': df_1hour.sma50,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'green'
            },
    'name': 'SMA 50'
}

trace5 = {
    'x': df_1hour.date,
    'y': df_1hour.sma100,
     'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'green'
            },
    'name': 'SMA 100'
}
trace6 = {
'x': df_1hour.date,
    'y': df_1hour.sma200,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'purple'
    },
    'name': 'SMA 200'
}
trace7 = {
'x': df_1hour.date,
    'y': df_1hour['BBL_14_2.0'],
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'white'
    },
    'name': 'BB'
}
trace8 = {
'x': df_1hour.date,
    'y': df_1hour['BBM_14_2.0'],
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'pink'
    },
    'name': 'BBM'
}
trace9 = {
'x': df_1hour.date,
    'y': df_1hour['BBU_14_2.0'],
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'white'
    },
    'name': 'BBU'
}

data = [trace1, trace2, trace3,trace4,trace5,trace6,trace7,trace8,trace9]

# Config graph layout



  #CHARTING
fig = go.Figure(data=data)

#fig.update_xaxes(rangeslider_visible=False)

fig.update_layout(
    title='test',
    autosize=False, width= 1000, height=1000,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="black", template ='plotly_dark',

    )



fig.add_scatter(x= Trade.Time, y= Trade['buy_signal'], mode="markers",
                marker=dict(size= 10, color="green"),
                name="Signal")

fig.add_scatter(x= Trade.Time, y= Trade['sell_signal'], mode="markers",
                marker=dict(size= 10, color="red"),
                name="Signal")
fig.show()






In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Convert the 'Holding Duration' column to timedelta format
df['Holding Duration'] = pd.to_timedelta(df['Holding Duration'])

# Convert holding duration to hours for better visualization
df['Holding Duration (hours)'] = df['Holding Duration'].dt.total_seconds() / 3600

# Plotting a histogram of holding durations
plt.hist(df['Holding Duration (hours)'], bins=10, edgecolor='black')

# Set labels and title
plt.xlabel('Holding Duration (hours)')
plt.ylabel('Frequency')
plt.title('Distribution of Holding Durations')

# Display the plot
plt.show()


In [ ]:
df = trade_report

# Calculate the total profit and total loss
profits = df[df['cumulative_total_net_of_fees'] > 0]['cumulative_total_net_of_fees']
losses = df[df['cumulative_total_net_of_fees'] < 0]['cumulative_total_net_of_fees']

total_profit = profits.sum()
total_loss = losses.abs().sum()

# Calculate the profit factor
profit_factor = total_profit / total_loss

print('profit factor: ',profit_factor)

In [ ]:
import pandas as pd

# Assuming you have a DataFrame named 'report_df' with the necessary columns

# Calculate the average gain (positive average) and average loss (negative average)
positive_average = report_df[report_df['cumulative_total_net_of_fees'] > 0]['cumulative_total_net_of_fees'].mean(numeric_only=True)
negative_average = report_df[report_df['cumulative_total_net_of_fees'] < 0]['cumulative_total_net_of_fees'].mean(numeric_only=True)

# Create a DataFrame for the report with an index
report = pd.DataFrame({'Average Gain': positive_average, 'Average Loss': negative_average}, index=[0])

print(report)




In [ ]:
import pandas as pd

# Assuming you have a DataFrame named 'report_df' with the necessary columns

# Count the number of winning trades
winning_trades = report_df[report_df['cumulative_total_net_of_fees'] > 0]

# Calculate the percentage of winning trades
winning_trade_percentage = (len(winning_trades) / len(report_df)) * 100

print(f"Percentage of winning trades: {winning_trade_percentage:.2f}%")


In [ ]:

# Assign colors based on the sign of the total net fees
colors = ['red' if fee < 0 else 'green' for fee in report_df['cumulative_total_net_of_fees']]
report_df = report_df.sort_values('cumulative_total_net_of_fees', ascending=False)
# Plot the chart
#plt.bar(report_df['Proprietary_ID'], report_df['cumulative_total_net_of_fees'], color=colors)
plt.bar(report_df['Proprietary_ID'], report_df['cumulative_total_net_of_fees'], color=colors)
plt.xlabel('Time')
plt.ylabel('cumulative_total_net_of_fees')
plt.title('Trade P&L Net Fees Report')
# Rotate x-axis labels diagonally
plt.xticks(rotation=45, ha='right')

plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you have a DataFrame named 'df' with columns 'date' and 'profits'

# Convert 'date' column to datetime type
df['date'] = pd.to_datetime(df['date'])

# Group the data by date and sum the profits
daily_profits = df.groupby(df['date'].dt.date)['profits'].sum()

# Create the line chart
plt.plot(daily_profits.index, daily_profits.values)
plt.xlabel('Date')
plt.ylabel('Profits')
plt.title('Daily Profits')
plt.xticks(rotation=45)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Remove "/USDT" from the 'Market' column
analyzed_df['Market'] = analyzed_df['Market'].str.replace('/USDT', '')
# Group the DataFrame by 'Trade ID' and calculate the sum of 'Total_net_fees'
trade_report = analyzed_df.groupby('Market')['Total_net_fees'].sum()

# Create a new DataFrame for the report
report_df = pd.DataFrame({'Total_net_fees': trade_report})

# Assign colors based on the sign of the total net fees
colors = ['red' if fee < 0 else 'green' for fee in report_df['Total_net_fees']]

# Plot the chart
plt.bar(report_df.index, report_df['Total_net_fees'], color=colors)
plt.xlabel('Proprietary_ID')
plt.ylabel('Total_net_fees')
plt.title('Trade Net Fees Report')
# Rotate x-axis labels diagonally
plt.xticks(rotation=45, ha='right')

plt.show()


In [ ]:
# Group the DataFrame by 'Proprietary_ID' and 'Market' and calculate the cumulative sum of 'Total Net of Fees'
trade_report = analyzed_df.groupby(['Proprietary_ID', 'Market'])['cumulative_total_net_of_fees'].cumsum()

# Create a new DataFrame for the report
report_df = pd.DataFrame({'Proprietary_ID': analyzed_df['Proprietary_ID'], 'Market': analyzed_df['Market'], 'cumulative_total_net_of_fees': trade_report})

# Remove duplicates from the report DataFrame
report_df.drop_duplicates(inplace=True)

# Sort the report DataFrame by 'Proprietary_ID'
report_df = report_df.sort_values(by='Proprietary_ID')

# Print the trade report
report_df


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

# Assuming your analyzed DataFrame is stored in the variable 'analyzed_df'
# Convert the 'Time' column to datetime if it's not already in the proper format


# Set the 'Time' column as the index of the DataFrame

# Calculate the cumulative sum of the 'TotalNetFees' column
equity_curve = analyzed_df['cumulative_total_net_of_fees']
# Set the 'date' column as the index


# Create a line chart of the return over time
plt.figure(figsize=(10, 6))
plt.plot(analyzed_df.index, equity_curve, marker='o', linestyle='-')
# Plot the cumulative equity curve



# Set the date label format
date_formatter = DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(date_formatter)
plt.xticks(rotation=45)

plt.xlabel('Time')
plt.ylabel('Equity')
plt.xlabel('Time')
plt.title('Cumulative Equity Curve')
plt.show()


In [ ]:
import ccxt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import date
import time

exchange = ccxt.kucoin()


for Exchange in analyzed_df['Exchange'].unique():

    Filtered = analyzed_df[(analyzed_df['Exchange'] == Exchange)].sort_values(by='Time')

    for Market in Filtered['Market'].unique():


        trade = analyzed_df[analyzed_df['Market'] ==Market].sort_values(by='Time')

        start_date = trade['UNIX'].min()

        try:
            ohlcv = exchange.fetch_ohlcv(Market, '1d', since=start_date)
            df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
            df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
            df['Market'] = Market

 final
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'])])

            fig.update_layout(
                title=Exchange +' '+ Market,
                autosize=False, width= 1000, height=1000,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="black", template ='plotly_dark',

                )

            fig.add_scatter(x= trade.UNIX, y= trade['buy_signal'], mode="markers",
                            marker=dict(size= 10, color="yellow"),
                            name="buy")

            fig.add_scatter(x= trade.UNIX, y= trade['sell_signal'], mode="markers",
                            marker=dict(size= 10, color="white"),
                            name="sell")


            fig.show()
            print(trade)
        except Exception as e:
            pass

Load Trades

In [ ]:
import ccxt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import date
exchange = ccxt.kucoin()
markets = df['Market'].unique()

P&L

when are set on the journal no right nonevents that are merging designns nominate lame of a, 73 majestic sccheduling of the vision mmight put persoonal vision that for the national# Strategy

In [ ]:
#@title Visualize { display-mode: "code" }

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import date

df = df[0: 912].copy()
#df=df.drop(columns=['level_0'])#!!!!!!!!!!bash
#df.reset_date(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=df.Timestamp,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])



fig.add_scatter(x= Trade.Time, y= Trade['buy_signal'], mode="markers",
                marker=dict(size= 10, color="green"),
                name="Signal")

fig.add_scatter(x= Trade.Time, y= Trade['sell_signal'], mode="markers",
                marker=dict(size= 10, color="red"),
                name="Signal")

fig.show()



In [ ]:
#@title Fundamentals

import requests
import datetime
import time
import json
from google.colab import data_table
from pycoingecko import CoinGeckoAPI
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pytrends.request import TrendReq


cg = CoinGeckoAPI()

CoinGecko_id = 'ethereum'


response = requests.get("https://api.coingecko.com/api/v3/coins/{}".format(CoinGecko_id))
output = response.text
json_parsed = json.loads(output)


x = json_parsed['market_data']['fully_diluted_valuation']

def fdv_none_val(x):
  """
    Calculating supply expansion multiple
    Requiring maximum supply and circulating supply
  """
  if x is None :
    fully_diluted_valuation = 0
  else :
    fully_diluted_valuation = json_parsed['market_data']['fully_diluted_valuation']

  return fully_diluted_valuation



def zero_div(x,y):
  """
  handling 0 division error

  """
  try:
      return x/y
  except ZeroDivisionError:
      return 0
links = 'Complete list: '+str(json_parsed['links'])
chat_url = 'Chat: '+str(json_parsed['links']['chat_url'])
CoinGecko_Link =  'CoinGecko: '+ "https://www.coingecko.com/en/coins/" + CoinGecko_id
Twitters = 'Twitter: '+str('https://twitter.com/')+str(json_parsed['links']['twitter_screen_name'])
homepage = 'Homepage: '+str(json_parsed['links']['homepage'])
subreddit_url =  'Reddit: ' +str(json_parsed['links']['subreddit_url'])
pd.set_option('display.width', 10)

print('')
print('WEB Links')
print('')
print(homepage)
print(Twitters)
print(subreddit_url)
print(chat_url)
print(CoinGecko_Link)

print(links)

print('')
print('MARKET DATA')
print('')


market_cap = json_parsed['market_data']['market_cap']['usd']
circulating_supply = json_parsed['market_data']['circulating_supply']
# circulating_supply.astype(int)
current_price = json_parsed['market_data']['current_price']['usd']
# current_price.astype(int)
fdv_to_tvl_rati = json_parsed['market_data']['fdv_to_tvl_ratio']
# fdv_to_tvl_ratio.astype(int)


#fully_diluted_valuation = json_parsed['market_data']['fully_diluted_valuation']['usd']

# fully_diluted_valuation
total_supply = json_parsed['market_data']['total_supply']
max_supply = json_parsed['market_data']['max_supply']
mcap_to_tvl_ratio = json_parsed['market_data']['mcap_to_tvl_ratio']
total_supply = json_parsed['market_data']['total_supply']
#total_value_locked = json_parsed['market_data']['total_value_locked']['usd']
total_volume = json_parsed['market_data']['total_volume']['usd']

float_rotation_24h = zero_div(total_volume , market_cap)
circulating_cap = current_price * json_parsed['market_data']['circulating_supply']
total_cap = current_price * total_supply


def max_cap(current_price, max_supply):
  """
    Calculation of maximum capitalization
     Requiring current price and maximum supply
  """
  if max_supply is not None :
    max_cap = current_price * max_supply
  else :
    max_cap = 0
  return max_cap

max_cap = max_cap(current_price, max_supply)



def max_supply_expansion_multiple(circulating_supply, max_supply):
  """
    Calculating supply expansion multiple
    Requiring maximum supply and circulating supply
  """
  if max_supply is None :
    max_supply_expansion_multiple = 0

  elif circulating_supply ==  0:
    max_supply_expansion_multiple == 0

  else :
    max_supply_expansion_multiple = max_supply  / circulating_supply

  return max_supply_expansion_multiple


max_supply_expansion_multiple = max_supply_expansion_multiple(circulating_supply, max_supply)

def total_supply_expansion_multiple(circulating_supply, total_supply):
  """
    Calculating supply expansion multiple
    Requiring maximum supply and circulating supply
  """
  if total_supply is None :
    total_supply_expansion_multiple = 0

  elif circulating_supply == 0:
    total_supply_expansion_multiple == 0

  else :
    total_supply_expansion_multiple = total_supply  / circulating_supply

  return total_supply_expansion_multiple


"""
def market_cap_dilution_ratio(fully_diluted_valuation, market_cap):

But is vital
    market_cap_dilution_ratio == 0

  elif fully_diluted_valuation or market_cap is 0:
    market_cap_dilution_ratio == 0

  elif fully_diluted_valuation or market_cap is {}:
    market_cap_dilution_ratio == 0

  else :
    market_cap_dilution_ratio =  fully_diluted_valuation / market_cap

  return market_cap_dilution_ratio


market_cap_dilution_ratio = market_cap_dilution_ratio(fully_diluted_valuation, market_cap)
"""

print('')

circulating_versus_max = circulating_supply/total_supply
dict_custom = {}
dict_custom['total_supply_expansion_multiple'] = total_supply_expansion_multiple
dict_custom['max_supply_expansion_multiple'] = max_supply_expansion_multiple
dict_custom['float_rotation_24h'] = float_rotation_24h

dict_custom['circulating_versus_max'] = circulating_versus_max


print(dict_custom)

df_fund_market = {}
df_fund_market['market_cap'] = market_cap
#Final been up what is on the national emergency, informative or what basket is something the P is the spaciousness up to Sarah  ankedat1['circulating_supply'] = circulating_supply
df_fund_market['current_price'] = current_price
# df_fund_market['fdv_to_tvl_ratio'] = fdv_to_tvl_ratio#
#df_fund_market['fully_diluted_valuation'] = fully_diluted_valuation
# ankedat1['max_supply'] = max_supply
df_fund_market['mcap_to_tvl_ratio'] = mcap_to_tvl_ratio
df_fund_market['total_supply'] = total_supply
#df_fund_market['total_value_locked'] = total_value_locked
df_fund_market['mcap_to_tvl_ratio'] = mcap_to_tvl_ratio
df_fund_market['total_volume'] = total_volume


print(df_fund_market)




print('          ')
print('PRICE ACTION')
price_change_percentage_7d = json_parsed['market_data']['price_change_percentage_7d']
price_change_percentage_14d = json_parsed['market_data']['price_change_percentage_14d']
price_change_percentage_30d = json_parsed['market_data']['price_change_percentage_30d']
price_change_percentage_60d = json_parsed['market_data']['price_change_percentage_60d']
price_change_percentage_200d = json_parsed['market_data']['price_change_percentage_200d']
price_change_percentage_1y = json_parsed['market_data']['price_change_percentage_1y']



dict_return = {}
dict_return['price_change_percentage_7d'] = price_change_percentage_7d
dict_return['price_change_percentage_14d'] = price_change_percentage_14d
dict_return['price_change_percentage_30d'] = price_change_percentage_30d
dict_return['price_change_percentage_60d'] = price_change_percentage_60d
dict_return['price_change_percentage_200d'] = price_change_percentage_200d
dict_return['price_change_percentage_1y'] = price_change_percentage_1y

print(dict_return)



category = json_parsed['categories']

print('')
print('CATEGORIES')
print(category)




description  = []
pd.set_option('display.width', 10)
pd.set_option("display.max_rows", None, "display.max_columns", None)
description = json_parsed['description']['en']

print('')
print('DESCRIPTION: ')
print(description)


links = json_parsed['links']

print('')
print('LIQUIDITY: ')
print('')

a = json_parsed['tickers']
data = pd.json_normalize(a,max_level=4)
df_fund = pd.DataFrame(data)

df_fund['pair'] = df_fund['base'] + df_fund['target']

df_fund = df_fund.sort_values("converted_volume.usd",ascending=False)
exchange =  []
exchange = df_fund['market.identifier'].to_list()

df_fund['liquidity_ranking'] = str(exchange)
df_fund['liquidity_ranking'] = df_fund['liquidity_ranking'].apply(eval)
df_fund['most_liquid_exchange'] = df_fund['liquidity_ranking'][1][0]
df_fund

# Trade Review

In [ ]:
#@title Results { display-mode: "code" }
from backtesting import Strategy
from backtesting import Backtest


dfpl = df[:].copy()
def SIGNAL():
    return dfpl.ordersignal

class MyStrat(Strategy):
    initsize = 0.99
    mysize = initsize

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2
        perc = 0.02


        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:
            sl1 = min(self.data.Low[-1],self.data.Low[-2])*(1-perc)
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:
            sl1 = sl1 = max(self.data.High[-1],self.data.High[-2])*(1+perc)
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/1, commission=.001)
stat = bt.run()
stat_=pd.DataFrame(stat)
stat_


In [ ]:
import pandas as pd


def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70

    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)

        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)

        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)


    def next(self):
        price = self.data.Close[-1]

        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):

            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)

        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

from backtesting.test import GOOG

backtest = Backtest(GOOG, System, commission=.002)
backtest.run()

In [ ]:
%%time

backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

In [ ]:
stats, heatmap = backtest.optimize(
    d_rsi=range(10, 35, 5),
    w_rsi=range(10, 35, 5),
    level=range(30, 80, 10),
    maximize='Equity Final [$]',
    max_tries=200,
    random_state=0,
    return_heatmap=True)

In [ ]:
heatmap

In [ ]:
hm = heatmap.groupby(['d_rsi', 'w_rsi']).mean().unstack()
hm

In [ ]:
%matplotlib inline

import seaborn as sns


sns.heatmap(hm[::-1], cmap='viridis')

In [ ]:
from backtesting.lib import plot_heatmaps


plot_heatmaps(heatmap, agg='mean')

In [ ]:
%%time

stats_skopt, heatmap, optimize_result = backtest.optimize(
    d_rsi=range(10, 35, 5),
    w_rsi=range(10, 35, 5),
    level=range(30, 80, 10),
    maximize='Equity Final [$]',
    method='skopt',
    max_tries=200,
    random_state=0,
    return_heatmap=True,
    return_optimization=True)

In [ ]:
heatmap.sort_values().iloc[-3:]

In [ ]:
from skopt.plots import plot_objective

_ = plot_objective(optimize_result, n_points=10)

In [ ]:
from skopt.plots import plot_evaluations

_ = plot_evaluations(optimize_result, bins=10)

In [ ]:
backtest.plot()

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()


bt = Backtest(dfpl, SmaCross, cash=100000, margin=1/1, commission=.001)
stat = bt.run()
stat_=pd.DataFrame(stat)
stat_


In [ ]:
heatmap

In [ ]:
heatmap.sort_values().iloc[-3:]

In [ ]:
#@title Chart { display-mode: "code" }
bt.plot()

#  Price Prediction

In [ ]:
from backtesting.test import EURUSD, SMA

data = EURUSD.copy()
data


In [ ]:
def BBANDS(data, n_lookback, n_std):
    """Bollinger bands indicator"""
    hlc3 = (data.High + data.Low + data.Close) / 3
    mean, std = hlc3.rolling(n_lookback).mean(), hlc3.rolling(n_lookback).std()
    upper = mean + n_std*std
    lower = mean - n_std*std
    return upper, lower


close = data.Close.values
sma10 = SMA(data.Close, 10)
sma20 = SMA(data.Close, 20)
sma50 = SMA(data.Close, 50)
sma100 = SMA(data.Close, 100)
upper, lower = BBANDS(data, 20, 2)

# Design matrix / independent features:

# Price-derived features
data['X_SMA10'] = (close - sma10) / close
data['X_SMA20'] = (close - sma20) / close
data['X_SMA50'] = (close - sma50) / close
data['X_SMA100'] = (close - sma100) / close

data['X_DELTA_SMA10'] = (sma10 - sma20) / close
data['X_DELTA_SMA20'] = (sma20 - sma50) / close
data['X_DELTA_SMA50'] = (sma50 - sma100) / close

# Indicator features
data['X_MOM'] = data.Close.pct_change(periods=2)
data['X_BB_upper'] = (upper - close) / close
data['X_BB_lower'] = (lower - close) / close
data['X_BB_width'] = (upper - lower) / close
data['X_Sentiment'] = ~data.index.to_series().between('2017-09-27', '2017-12-14')

# Some datetime features for good measure
data['X_day'] = data.index.dayofweek
data['X_hour'] = data.index.hour

data = data.dropna().astype(float)

In [ ]:
import numpy as np


def get_X(data):
    """Return model design matrix X"""
    return data.filter(like='X').values


def get_y(data):
    """Return dependent variable y"""
    y = data.Close.pct_change(48).shift(-48)  # Returns after roughly two days
    y[y.between(-.004, .004)] = 0             # Devalue returns smaller than 0.4%
    y[y > 0] = 1
    y[y < 0] = -1
    return y


def get_clean_Xy(df):
    """Return (X, y) cleaned of NaN values"""
    X = get_X(df)
    y = get_y(df).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X, y = get_clean_Xy(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=0)

clf = KNeighborsClassifier(7)  # Model the output based on 7 "nearest" examples
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

_ = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred}).plot(figsize=(15, 2), alpha=.7)
print('Classification accuracy: ', np.mean(y_test == y_pred))

In [ ]:
%%time

from backtesting import Backtest, Strategy

N_TRAIN = 400


class MLTrainOnceStrategy(Strategy):
    price_delta = .004  # 0.4%

    def init(self):
        # Init our model, a kNN classifier
        self.clf = KNeighborsClassifier(7)

        # Train the classifier in advance on the first N_TRAIN examples
        df = self.data.df.iloc[:N_TRAIN]
        X, y = get_clean_Xy(df)
        self.clf.fit(X, y)

        # Plot y for inspection
        self.I(get_y, self.data.df, name='y_true')

        # Prepare empty, all-NaN forecast indicator
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self):
        # Skip the training, in-sample data
        if len(self.data) < N_TRAIN:
            return

        # Proceed only with out-of-sample data. Prepare some variables
        high, low, close = self.data.High, self.data.Low, self.data.Close
        current_time = self.data.index[-1]

        # Forecast the next movement
        X = get_X(self.data.df.iloc[-1:])
        forecast = self.clf.predict(X)[0]

        # Update the plotted "forecast" indicator
        self.forecasts[-1] = forecast

        # If our forecast is upwards and we don't already hold a long position
        # place a long order for 20% of available account equity. Vice versa for short.
        # Also set target take-profit and stop-loss prices to be one price_delta
        # away from the current closing price.
        upper, lower = close[-1] * (1 + np.r_[1, -1]*self.price_delta)

        if forecast == 1 and not self.position.is_long:
            self.buy(size=.2, tp=upper, sl=lower)
        elif forecast == -1 and not self.position.is_short:
            self.sell(size=.2, tp=lower, sl=upper)

        # Additionally, set aggressive stop-loss on trades that have been open
        # for more than two days
        for trade in self.trades:
            if current_time - trade.entry_time > pd.Timedelta('2 days'):
                if trade.is_long:
                    trade.sl = max(trade.sl, low)
                else:
                    trade.sl = min(trade.sl, high)


bt = Backtest(data, MLTrainOnceStrategy, commission=.0002, margin=.05)
bt.run()

In [ ]:
bt.plot()

In [ ]:
%%time

class MLWalkForwardStrategy(MLTrainOnceStrategy):
    def next(self):
        # Skip the cold start period with too few values available
        if len(self.data) < N_TRAIN:
            return

        # Re-train the model only every 20 iterations.
        # Since 20 << N_TRAIN, we don't lose much in terms of
        # "recent training examples", but the speed-up is significant!
        if len(self.data) % 20:
            return super().next()

        # Retrain on last N_TRAIN values
        df = self.data.df[-N_TRAIN:]
        X, y = get_clean_Xy(df)
        self.clf.fit(X, y)

        # Now that the model is fitted,
        # proceed the same as in MLTrainOnceStrategy
        super().next()


bt = Backtest(data, MLWalkForwardStrategy, commission=.0002, margin=.05)
bt.run()

In [ ]:
#@title
# Just be sure any changes have been committed or they will be lost.
conn_SQLite.close()